# Build AI🤖 Agents Using LangGraph - Adding Memory to the Agent 🚀

In [1]:
from typing_extensions import TypedDict, Annotated
from typing import Any
from langchain_core.messages import HumanMessage, AnyMessage, SystemMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from dotenv import load_dotenv

load_dotenv()


# Define the state
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


# Tool: Person Information
def get_person_details(person_name: str) -> str:
    """Retrieve details of a person."""
    return f"{person_name} is a DevOps Engineer."


# Tool: Location Information
def get_person_location(person_name: str) -> str:
    """Retrieve location of a person."""
    return f"{person_name} lives in Bangalore."


# Initialize the Groq chat model
llm = ChatGroq(model="llama-3.3-70b-versatile")

# Bind tools to the Groq model
tools = [get_person_details, get_person_location]
llm_with_tools = llm.bind_tools(tools)


# System message
sys_msg = SystemMessage(
    content="You are a helpful assistant that provides accurate responses based on the given tools.")


# Define the assistant node
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


# Build the graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Adding Edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

# Example without memory
react_graph = builder.compile()

In [2]:
from langgraph.checkpoint.memory import MemorySaver 
memory = MemorySaver() 
react_graph_memory = builder.compile(checkpointer=memory)

In [3]:
config = {"configurable": {"thread_id": "1"}}

In [4]:
from typing_extensions import TypedDict, Annotated
from typing import Any
from langchain_core.messages import HumanMessage, AnyMessage, SystemMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

from dotenv import load_dotenv

load_dotenv()


# Define the state
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


# Tool: Person Information
def get_person_details(person_name: str) -> str:
    """Retrieve details of a person."""
    return f"{person_name} is a DevOps Engineer."


# Tool: Location Information
def get_person_location(person_name: str) -> str:
    """Retrieve location of a person."""
    return f"{person_name} lives in Bangalore."


# Initialize the Groq chat model
llm = ChatGroq(model="llama-3.3-70b-versatile")

# Bind tools to the Groq model
tools = [get_person_details, get_person_location]
llm_with_tools = llm.bind_tools(tools)


# System message
sys_msg = SystemMessage(
    content="You are a helpful assistant that provides accurate responses based on the given tools.")


# Define the assistant node
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


# Build the graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Adding Edge
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Test interaction with memory
# Thread ID for maintaining context
config = {"configurable": {"thread_id": "1"}}


messages = [HumanMessage(
    content="Tell me about Harsha?")]
result = react_graph.invoke({"messages": messages}, config)


messages = [HumanMessage(
    content="Is Harsha a Devops Enginer?")]
result = react_graph.invoke({"messages": messages}, config)
for message in result['messages']:
    message.pretty_print()

================================ Human Message =================================

Is Harsha a Devops Enginer?
================================== Ai Message ==================================
Tool Calls:
  get_person_details (call_sptf)
 Call ID: call_sptf
  Args:
    person_name: Harsha
================================= Tool Message =================================
Name: get_person_details

Harsha is a DevOps Engineer.
================================== Ai Message ==================================
Tool Calls:
  get_person_details (call_ch68)
 Call ID: call_ch68
  Args:
    person_name: Harsha
================================= Tool Message =================================
Name: get_person_details

Harsha is a DevOps Engineer.
================================== Ai Message ==================================

I am not able to confirm if Harsha is a DevOps Engineer.
